Rebuilding Steps, Example Network, Integrated in SNMan 'Multi' Rebuild
====================
xxx

In [1]:
import warnings
warnings.filterwarnings('ignore')

import copy
import os
import sys
import networkx as nx
import numpy as np
import shutil

import snman
from snman.constants import *
from snman import osmnx_customized as oxc
from snman import *

PERIMETER = 'db'

data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
export_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

Loading data
------------
Loads the prepared simplified street graph, as well as other datasets such as perimeters, rebuilding regions and measurement regions

In [2]:
print('Load street graph')
G = snman.io.load_street_graph(
    os.path.join(process_path, 'street_graph_edges.gpkg'),
    os.path.join(process_path, 'street_graph_nodes.gpkg'),
    crs=CRS_internal
)

street_graph.organize_edge_directions(G)

Load street graph


In [3]:
widths = snman.io.import_geofile_to_gdf('C:/Users/lballo/polybox/BUSINESS/PROJECTS/01 Dornbirn GVK/DATA/strassennetz_mit_breiten_bereinigt.gpkg')
widths['width'] = widths['fb_1_avgw']
widths

,gid,ogc_fid,sde_id,subnetid,fow,frc,edgecat,edgecatn,erhalter,lstart,...,fb_2_typ,fb_2_avgw,fb_3_typ,fb_3_avgw,fb_4_typ,fb_4_avgw,fb_5_typ,fb_5_avgw,geometry,width
0,50908,NaN,1608366,14103,003 - Ungeteilte Fahrbahn,006 - Sammelstraßen,G,Gemeindestraße,Dornbirn,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (2773775.101 1255456.592, 2773791.6...",4.0
1,51224,NaN,1608986,14103,003 - Ungeteilte Fahrbahn,011 - Wirtschaftsweg,WW,Wirtschaftsweg,Dornbirn,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (2773705.057 1257139.235, 2773712.0...",3.0
2,51241,NaN,1306148,14103,003 - Ungeteilte Fahrbahn,107 - Gebäudezufahrt,G,Gemeindestraße,Dornbirn,0,...,Gehweg,1.5,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (2772700.454 1254026.528, 2772713.4...",5.0
3,51279,NaN,1609027,14103,003 - Ungeteilte Fahrbahn,007 - Straßen der internen Erschließung,G,Gemeindestraße,Dornbirn,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (2773217.002 1254243.850, 2773248.4...",5.0
4,51286,NaN,1609026,14103,003 - Ungeteilte Fahrbahn,007 - Straßen der internen Erschließung,G,Gemeindestraße,Dornbirn,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (2773080.299 1254165.433, 2773081.7...",5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2568,50065,NaN,1267363,14103,014 - Fußgängerzone,008 - Sonstige Straße,G,Gemeindestraße,Dornbirn,0,...,Gehweg,1.5,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (2773855.422 1253887.818, 2773858.6...",5.0
2569,50212,NaN,1616943,14103,003 - Ungeteilte Fahrbahn,008 - Sonstige Straße,G,Gemeindestraße,Dornbirn,0,...,Gehweg,1.5,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (2773527.309 1254360.300, 2773532.9...",7.0
2570,50213,NaN,1260946,14103,003 - Ungeteilte Fahrbahn,005 - Straßen des innerörtlichen Netzes,G,Gemeindestraße,Dornbirn,0,...,Gehweg,1.5,Radfahrstreifen,1.0,NaN,NaN,NaN,NaN,"LINESTRING (2773133.525 1253589.470, 2773062.7...",9.0
2571,50296,NaN,1616286,14103,003 - Ungeteilte Fahrbahn,007 - Straßen der internen Erschließung,G,Gemeindestraße,Dornbirn,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (2774473.806 1252622.646, 2774471.7...",3.5


In [4]:
snman.enrichment.match_width(
    G, widths, 
    max_dist=200, max_dist_init=500, max_lattice_width=5
)

In [5]:
if 1:
    print('export G')
    snman.io.export_street_graph(
        G,
        os.path.join(process_path, 'G_edges.gpkg'),
        os.path.join(process_path, 'G_nodes.gpkg')
    )

export G
